In [1]:
!pip install langchain faiss-cpu tiktoken
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install --quiet langchain_community langchainhub sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.1 MB/s eta 0:00:00


In [3]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 13.2 MB/s eta 0:00:00


In [4]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
import os
from google.colab import userdata
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HUGGINGFACEHUB_API_TOKEN')

In [6]:
os.environ["LANGCHAIN_WANDB_TRACING"]  = "true"


In [24]:
import wandb

In [29]:
loader = PyPDFLoader("/content/kkk.pdf")

ValueError: File path /content/wandb is not a valid file or url

In [9]:
doc = loader.load()

In [10]:
doc

[Document(metadata={'source': '/content/kkk.pdf', 'page': 0}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)

In [12]:
text = text_splitter.split_documents(doc)

In [13]:
text[0]

Document(metadata={'source': '/content/kkk.pdf', 'page': 0}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto')

In [14]:
## Embedding Using Huggingface
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",      #sentence-transformers/all-MiniLM-l6-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}

)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
vecstore = FAISS.from_documents(text, huggingface_embeddings)


The Flat in the context of FAISS.from_documents(text, huggingface_embeddings, "Flat") refers to the type of index used by FAISS (Facebook AI Similarity Search) for storing and searching embeddings. FAISS offers various types of indexes that are optimized for different performance and accuracy trade-offs. Here's an explanation of the Flat index and its alternatives:

### Flat Index

Flat Index (IndexFlatL2 or IndexFlatIP):
Description: The Flat index, also known as the brute-force index, stores all the vectors as they are and performs a linear scan to find the nearest neighbors. This means that for each query, the distance to every vector in the index is computed to determine the closest matches.

Advantages:
Simple to use and implement.
Provides the most accurate results because it considers all vectors.
No loss of information since no approximation is involved.

Disadvantages:
Computationally expensive, especially for large datasets, as it requires a full scan of all vectors.
Memory-intensive since it needs to store all vectors in memory.
Use Cases: Suitable for small to medium-sized datasets where high accuracy is required, and computational resources are not a bottleneck.

## Alternatives to Flat Index

FAISS offers several other types of indexes, each with different trade-offs in terms of speed, memory usage, and accuracy:

### IVF (Inverted File) Indexes:

IndexIVFFlat: Uses a clustering-based approach where the dataset is divided into clusters, and only a subset of clusters is searched for each query.
Advantages: Faster than Flat for large datasets because it reduces the number of comparisons.
Disadvantages: Can be less accurate due to approximations in cluster selection.

### PQ (Product Quantization) Indexes:

IndexPQ:
Compresses the vectors into smaller representations, allowing for faster search and reduced memory usage.

Advantages: Significantly reduces memory footprint and improves search speed.

Disadvantages: Lower accuracy due to lossy compression.

### HNSW (Hierarchical Navigable Small World) Index:

IndexHNSW:
Uses a graph-based approach for fast approximate nearest neighbor search.

Advantages: Provides a good balance between speed and accuracy, especially for high-dimensional data.

Disadvantages: More complex to implement and tune.

### LSH (Locality-Sensitive Hashing) Index:

IndexLSH: Uses hash functions to map similar vectors to the same buckets, reducing the search space.

Advantages: Very fast for high-dimensional data.

Disadvantages: Can be less accurate due to hashing approximations.

### IVF-PQ (Inverted File with Product Quantization):

IndexIVFPQ: Combines inverted file indexing with product quantization to balance memory usage and search speed.

Advantages: Efficient for large datasets with reasonable accuracy.

Disadvantages: Requires careful tuning of parameters for optimal performance.

Example Usage
To use different types of FAISS indexes, you can specify the index type when creating the FAISS index. Here’s an example of how to create and use some of these indexes:

In [16]:
vecstore.save_local("/content")

In [17]:
from langchain.chains import RetrievalQA

In [18]:
from langchain_community.llms import HuggingFaceHub

hf=HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1,"max_length":500}

)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(


In [19]:
retriever = vecstore.as_retriever()

In [20]:
chain = RetrievalQA.from_chain_type(
    retriever = retriever,
    llm  = hf,
    chain_type = "stuff",
    verbose = True

)

In [21]:
chain

RetrievalQA(verbose=True, combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"), llm=HuggingFaceHub(client=<InferenceClient(model='mistralai/Mistral-7B-v0.1', timeout=None)>, repo_id='mistralai/Mistral-7B-v0.1', task='text-generation', model_kwargs={'temperature': 0.1, 'max_length': 500})), document_variable_name='context'), retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7d4abdf19510>))

In [22]:
print(chain.invoke("What is Attention Mechanism ?"))



> Entering new RetrievalQA chain...

> Finished chain.
{'query': 'What is Attention Mechanism ?', 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nreduced to a constant number of operations, albeit at the cost of reduced effective resolution due\nto averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as\ndescribed in section 3.2.\nSelf-attention, sometimes called intra-attention is an attention mechanism relating different positions\nof a single sequence in order to compute a representation of the sequence. Self-attention has been\n\n.\n<EOS>\n<pad>\n<pad>\n<pad>\n<pad>\n<pad>\n<pad>\nFigure 3: An example of the attention mechanism following long-distance dependencies in the\nencoder self-attention in layer 5 of 6. Many of the attention heads attend to a distant dependency of\nthe verb ‘making’, completing the phrase ‘ma